Importing all Libraries needed to train as well as get information from the data base

In [9]:
import pandas as pd
import logging
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pytds


Connecting to database to retrieve student and training resumes

In [99]:
connect_string = ""
try:
    con = pytds.connect(server = '2605:ad80:30:5058:84ea:3de3:39a1:f11', database = 'TBDraft', user = 'admin1', password='TBuilder')
    cursor = con.cursor()
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

INFO - 23:20:29: Opening socket to 2605:ad80:30:5058:84ea:3de3:39a1:f11:1433
INFO - 23:20:29: Sending PRELOGIN lib_ver=10b0000 enc_flag=2 inst_name=b'MSSQLServer' mars=False
INFO - 23:20:29: Got PRELOGIN response crypt=2 mars=0
INFO - 23:20:29: Sending LOGIN tds_ver=74000004 bufsz=4096 pid=4264 opt1=f0 opt2=2 opt3=8 cli_tz=-360 cli_lcid=1033 cli_host=DESKTOP-5D53OF5 lang= db=TBDraft
INFO - 23:20:29: switched to database TBDraft
INFO - 23:20:29: switched collation to Collation(lcid=1033, sort_id=52, ignore_case=True, ignore_accent=False, ignore_width=True, ignore_kana=True, binary=False, binary2=False, version=0)
INFO - 23:20:29: switched language to us_english
INFO - 23:20:29: Got LOGINACK tds_ver=74000004 srv_name=Microsoft SQL Server   srv_ver=f0007d0
INFO - 23:20:29: Sending BEGIN TRAN il=0


 is working


In [100]:
##After connecting to database extract preprocessed information
#These are the base training resumes
df_training = pd.read_sql_query("SELECT ALL * FROM TrainingResumes", con)
#There are the processed student resumes
df_students = pd.read_sql_query("SELECT ALL * FROM StudentInformation", con)
con.close()
display(df_training)
display(df_students)


INFO - 23:20:40: Sending query SELECT ALL * FROM TrainingResumes
INFO - 23:20:40: Sending query SELECT ALL * FROM StudentInformation


,keyword,dataText
0,Electronics_Engineer,andre salazar fort worth tx email talent ele...
1,Electronics_Engineer,arturo cervante mission tx email seek employ...
2,Electronics_Engineer,juan avila charleston sc email experience el...
3,Electronics_Engineer,travis ray engineer product designer develop...
4,Embedded_Systems,daniel koh forney tx email software developm...
...,...,...
101,Software_Engineer,kai schmidt software engineer citrus height ...
102,Software_Engineer,nathan nguyen irvine email work experience s...
103,Software_Engineer,peter pagano bohemia ny email authorize work...
104,Software_Engineer,steven nichol c c software engineer grand ra...


,ID,StudentName,GPA,Email,StudentResume,ProcessedData,PhoneNumber
0,10,None,None,None,b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,None,None
1,11,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None
2,12,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None
3,13,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None
4,14,None,None,None,b'%PDF-1.4\n%\xf6\xe4\xfc\xdf\n1 0 obj\n<<\n/T...,None,None


In [42]:
df = pd.DataFrame(columns=['Keyword', 'Data'])
df['Keyword'] = df_training['keyword'] #+ df_students['']
df['Data'] = df_training['dataText'] #+ df_students['']
df.Data[105]

'  xueqiang pan highland ut email skilled innovative senior software engineer year experience enthusiastic write readable testable beautiful code build robust high performing scalable system willing relocate work experience senior software engineer adobe lehi ut december present globalization backend service development node js cassandra design implement foundational part backend service e g authentication middleware log error handling automation testing design implement core apis lead team development discuss feature implementation review prs designing database table follow node js good practice host server aw follow ci cd deployment process globalization frontend portal development react react spectrum design implement foundational part frontend portal e g redux setup routing api module message module n implement core feature component lead team development discuss ui mockup ux designer assign development task review prs globalization dashboard development node js mysql tableau colle

For Now we use tika to parse resumes and clean them

In [13]:
sent = [row.split() for row in df['Data']] #Change change ['clean'] to whatever we actually label the columns as
documents = [TaggedDocument(doc, tags=[df.Keyword[i]]) for i, doc in enumerate(sent)]

Training the model and saving it

In [55]:
threads = multiprocessing.cpu_count()
model = Doc2Vec(documents, 
                dm = 0,
                dbow_words=1, 
                alpha=0.03, 
                vector_size=30, 
                min_alpha = 0.0007, 
                min_count = 15, 
                negative=15,
                workers=threads-1)
model.save('d2v_model')

INFO - 16:46:58: collecting all words and their counts
INFO - 16:46:58: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 16:46:58: collected 7058 word types and 6 unique tags from a corpus of 106 examples and 56454 words
INFO - 16:46:58: Creating a fresh vocabulary
INFO - 16:46:58: Doc2Vec lifecycle event {'msg': 'effective_min_count=15 retains 727 unique words (10.30036837631057%% of original 7058, drops 6331)', 'datetime': '2021-11-28T16:46:58.776397', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
INFO - 16:46:58: Doc2Vec lifecycle event {'msg': 'effective_min_count=15 leaves 38842 word corpus (68.80291919084564%% of original 56454, drops 17612)', 'datetime': '2021-11-28T16:46:58.776397', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'p

In [95]:
tokens ="nlp, machine learn, analytic".split()
new_vec = model.infer_vector(tokens)
sims = model.dv.most_similar(positive=[new_vec])
display(sims)
model.predict_output_word('fpga')
model.similarity_unseen_docs(doc_words1=df.Data[59].split(), doc_words2="nlp, machine learn, analytic".split())

[('Machine_Learning', 0.7223070859909058),
 ('RF_Analog_Engineer', 0.5940945744514465),
 ('Embedded_Systems', 0.5620774030685425),
 ('Electronics_Engineer', 0.5261824727058411),
 ('Software_Engineer', 0.5148240327835083),
 ('Power_Systems_Engineer', 0.5105586647987366)]

0.6295652

In [87]:
df.Data[59].split()

['shreya',
 'ayyangar',
 'data',
 'scientist',
 'lindstrom',
 'mn',
 'email',
 'passionate',
 'datum',
 'engineer',
 'year',
 'analytic',
 'experience',
 'bfsi',
 'domain',
 'excellent',
 'team',
 'building',
 'management',
 'skill',
 'work',
 'artificial',
 'intelligence',
 'intern',
 'win',
 'st',
 'place',
 'unicef',
 'ias',
 'data',
 'analytic',
 'hackathon',
 'design',
 'interactive',
 'dashboard',
 'showcase',
 'important',
 'child',
 'development',
 'indicator',
 'factor',
 'country',
 'skilled',
 'machine',
 'learn',
 'natural',
 'language',
 'processing',
 'datum',
 'mining',
 'datum',
 'modeling',
 'datum',
 'visualizing',
 'datum',
 'management',
 'relational',
 'non',
 'relational',
 'database',
 'business',
 'analytic',
 'graduate',
 'ms',
 'data',
 'science',
 'specialization',
 'demonstrate',
 'history',
 'deliver',
 'large',
 'complex',
 'project',
 'passion',
 'technology',
 'positive',
 'impact',
 'create',
 'business',
 'transformation',
 'process',
 'improvement',
 